In [196]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

import numpy as np
from random import shuffle

from sklearn import svm
from sklearn import ensemble
from sklearn import model_selection as ms
from sklearn.metrics import mean_squared_error

# Preparation of datasets

In [151]:
#read data - ChemblID, canonical smiles, standard value and standard unit
data_file = open("./data/beta_amyloidA4_protein_IC50.tsv")
data_file.readline() #read the header (remove it)
def splitter(line):
    spls = line.strip().split("\t")
    if spls[14] != "" and spls[15] != "":
        return (spls[0],Chem.MolFromSmiles(spls[10]),float(spls[14]), spls[15])
mols = [splitter(line) for line in data_file]

In [152]:
len(mols)

854

In [153]:
mols = [m for m in mols if m and m[3] =="nM"]
len(mols)

785

In [176]:
active = [m for m in mols if m[2] < 1000]
active = [a + (1,) for a in active] #add 1 as a clasifier for active
shuffle(active)
len(active), active[:5]

(385,
 [('CHEMBL1744460', <rdkit.Chem.rdchem.Mol at 0x2416c265a80>, 19.0, 'nM', 1),
  ('CHEMBL236182', <rdkit.Chem.rdchem.Mol at 0x2416c2b3f80>, 13.0, 'nM', 1),
  ('CHEMBL492024', <rdkit.Chem.rdchem.Mol at 0x2416c2d7b20>, 820.0, 'nM', 1),
  ('CHEMBL2179242', <rdkit.Chem.rdchem.Mol at 0x2416c2b93f0>, 900.0, 'nM', 1),
  ('CHEMBL3667030', <rdkit.Chem.rdchem.Mol at 0x2416c2b90d0>, 500.0, 'nM', 1)])

In [181]:
inactive = [m for m in mols if m[2] >= 10000]
inactive = [i + (0,) for i in inactive] #add 0 as a clasifier for non active
len(inactive)

239

In [200]:
splitI = int(len(inactive)*0.7)
splitA = int(len(active)*0.7)

training = active[0:splitA] + inactive[0:splitI]
testing = active[splitA:] + inactive[splitI:]
len(training), len(testing)

(436, 188)

## Model training

In [199]:
shuffle(training)

In [209]:
#use 5-fold ross alidation!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!